In [1]:
import pickle
import pygame
import numpy as np
import matplotlib.pyplot as plt
import sys
import random

pygame 2.5.2 (SDL 2.28.3, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Brain:
    def __init__(self):

        i=5
        h=3
        o=1
        
        self.w1 = np.random.rand(i, h)*2-1
        self.w2 = np.random.rand(h, o)*2-1
        
    def forward(self, inputs):
        hidden_inputs = np.dot(inputs, self.w1)
        hidden_outputs = self._sigmoid(hidden_inputs)
        
        final_inputs = np.dot(hidden_outputs, self.w2)
        final_outputs = self._sigmoid(final_inputs)
        
        return final_outputs
    
    def _sigmoid(self, x):
        x = np.clip(x, -500, 500)  # Clip values to avoid overflow in exp
        return np.where(
            x >= 0,
            1 / (1 + np.exp(-x)),
            np.exp(x) / (1 + np.exp(x))
    )
    

In [3]:
with open('brains.pkl', 'rb') as file:
    brains = pickle.load(file)

In [11]:
# WITH PLAYER vs ALL THE BIRDS OF LAST GEN

# Constants
WIDTH, HEIGHT = 576, 800
GRAVITY = 0.25
BIRD_JUMP = 5
PIPE_GAP = 200
PIPE_VELOCITY = 3
BIRD_WIDTH, BIRD_HEIGHT = 64, 48
PIPE_WIDTH, PIPE_HEIGHT = 100, 800
BACKGROUND_COLOR = (0, 102, 204)
BIRD_COLOR = (255, 255, 0)
PIPE_COLOR = (0, 153, 51)
pop=100
alicount=pop-1

# Initialize Pygame
pygame.init()

# Set up the display
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()

# Load assets
bird_img = pygame.image.load('birdss.png').convert_alpha()
bird_img = pygame.transform.scale(bird_img, (BIRD_WIDTH, BIRD_HEIGHT))
bird_img2 = pygame.image.load('bird.png').convert_alpha()
bird_img2 = pygame.transform.scale(bird_img2, (BIRD_WIDTH, BIRD_HEIGHT))
pipe_img = pygame.image.load('bird1.png').convert_alpha()
pipe_img = pygame.transform.scale(pipe_img, (PIPE_WIDTH, PIPE_HEIGHT))

# Bird class
class Bird:
    def __init__(self):
        self.x = WIDTH // 3
        self.y = HEIGHT // 2
        self.velocity = 0
        
        self.alive = True
        self.sc = 0
    
    def jump(self):
        self.velocity = -BIRD_JUMP
    
    def update(self):
        self.velocity += GRAVITY
        self.y += self.velocity
    
    def draw(self):
        screen.blit(bird_img, (self.x, self.y))
        
    def draw2(self):
        screen.blit(bird_img2, (self.x, self.y))

# Pipe class
class Pipe:
    def __init__(self):
        self.x = WIDTH
        self.gap = random.randint(150, 170)
        self.top_height = random.randint(100, HEIGHT - self.gap - 100)
        self.bottom_height = HEIGHT - self.gap - self.top_height
    
    def move(self):
        self.x -= PIPE_VELOCITY
    
    def draw(self):
        screen.blit(pipe_img, (self.x, 0), (0, 0, PIPE_WIDTH, self.top_height))
        screen.blit(pipe_img, (self.x, self.top_height + self.gap), (0, self.top_height + self.gap, PIPE_WIDTH, self.bottom_height))

# Function to check collision
def check_collision(bird, pipes):
    if bird.y <= 0 or bird.y >= HEIGHT - BIRD_HEIGHT:
        return True
    for pipe in pipes:
        if bird.x + BIRD_WIDTH >= pipe.x and bird.x <= pipe.x + PIPE_WIDTH:
            if bird.y <= pipe.top_height or bird.y + BIRD_HEIGHT >= pipe.top_height + pipe.gap:
                return True
    return False

# Main function
def main():
    global BACKGROUND_COLOR
    global PIPE_VELOCITY
    global alicount
    birds = [Bird() for _ in range(pop)]
    pipes = [Pipe()]
    score = 0
    c_pressed=False
    
    default_font = pygame.font.Font(None, 50)
    default_font1 = pygame.font.Font(None, 40)
    default_font2 = pygame.font.Font(None, 20)
    
    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_c:
                    c_pressed = True
                if event.key == pygame.K_SPACE:
                    birds[0].jump()
            if event.type == pygame.KEYUP:
                if event.key == pygame.K_c:
                    c_pressed = False

        # Check if the spacebar is pressed and make the bird jump
        if c_pressed:
            birds[0].jump()
        
        for ib,bird in enumerate(birds):
            if(bird.alive and ib>=1):
                bo = brains[ib].forward((np.array([birds[ib].y, birds[ib].velocity, pipes[0].x, pipes[0].gap, pipes[0].top_height])).reshape(1,5))
                if(bo>0.5):
                    birds[ib].jump()
        
        
        
        # Update bird and pipes
        for bird in birds:
            if(bird.alive):
                bird.update()
        for pipe in pipes:
            pipe.move()
        
        for bird in birds:
            # Check if bird collides with pipes
            if check_collision(bird, pipes):
                if(bird.alive):
                    if(bird==birds[0]):
                        print("Game Over! U worse than AI, Score:", score)
                        for ff in range(360):
                            text_surface = default_font.render('Congrats! U worse than AI', False, (255, 255, 255))
                            text_rect = text_surface.get_rect(center=(WIDTH//2, HEIGHT//2))
                            screen.blit(text_surface, text_rect)
                            text_surface = default_font.render(f'Score: {score}', False, (255, 255, 255))
                            text_rect = text_surface.get_rect(center=(WIDTH//2, HEIGHT//2+50))
                            screen.blit(text_surface, text_rect)
                            pygame.display.update()
                            clock.tick(120)
                        pygame.quit()
                        sys.exit()
                    elif(alicount==1):
                        print("You WIN! Score:", score)
                        for ff in range(360):
                            text_surface = default_font.render('You WIN!', False, (255, 255, 255))
                            text_rect = text_surface.get_rect(center=(WIDTH//2, HEIGHT//2))
                            screen.blit(text_surface, text_rect)
                            text_surface = default_font.render(f'Score: {score}', False, (255, 255, 255))
                            text_rect = text_surface.get_rect(center=(WIDTH//2, HEIGHT//2+50))
                            screen.blit(text_surface, text_rect)
                            pygame.display.update()
                            clock.tick(120)
                        pygame.quit()
                        sys.exit()
                    else:
                        alicount-=1
                        bird.alive = False
                        
        screen.fill(BACKGROUND_COLOR)

        
        # Check if bird passes through pipes
        if pipes[0].x <= bird.x:
            score += 1
            pipes.pop(0)
            pipes.append(Pipe())
            # A bird dies every 4 score
            if(score%2==0):
                for bird in birds:
                    if(bird.alive and bird!=birds[0]):
                        bird.alive=False
                        alicount-=1
                        break
        
        # Draw bird and pipes
        for bird in birds:
            if(bird.alive and bird!=birds[0]):
                bird.draw()
        birds[0].draw2()
        for pipe in pipes:
            pipe.draw()
            
        text_surface = default_font1.render(f'Bots alive: {alicount}', False, (255, 255, 255))
        text_rect = text_surface.get_rect(center=(3.2*WIDTH//4, 40))
        screen.blit(text_surface, text_rect)
        
        text_surface = default_font1.render('Macro Jump: hold C', False, (255, 255, 255))
        text_rect = text_surface.get_rect(center=(3*WIDTH//4, HEIGHT-30))
        screen.blit(text_surface, text_rect)
        text_surface = default_font1.render('Single Jump: Space', False, (255, 255, 255))
        text_rect = text_surface.get_rect(center=(3*WIDTH//4, HEIGHT-60))
        screen.blit(text_surface, text_rect)
        
        text_surface = default_font1.render(f'Score: {score}', False, (255, 255, 255))
        text_rect = text_surface.get_rect(center=(90, 40))
        screen.blit(text_surface, text_rect)
        
        text_surface = default_font2.render('*A bot dies every 2 points', False, (255, 255, 255))
        text_rect = text_surface.get_rect(center=(90, HEIGHT-30))
        screen.blit(text_surface, text_rect)
        
        if(alicount==0):
            print("You WIN! Score:", score)
            for ff in range(360):
                text_surface = default_font.render('You WIN!', False, (0, 0, 0))
                text_rect = text_surface.get_rect(center=(WIDTH//2, HEIGHT//2))
                screen.blit(text_surface, text_rect)
                text_surface = default_font.render(f'Score: {score}', False, (0, 0, 0))
                text_rect = text_surface.get_rect(center=(WIDTH//2, HEIGHT//2+50))
                screen.blit(text_surface, text_rect)
                pygame.display.update()
                clock.tick(120)
            pygame.quit()
            
            sys.exit()
        
#         if(score%5==3):
#             PIPE_VELOCITY+=0.5
#             score+=1
#             a=np.random.random_integers(low=0, high=255, size=3)
#             BACKGROUND_COLOR=a
                
    
        pygame.display.update()
        clock.tick(80)

if __name__ == "__main__":
    main()

SystemExit: 